# Bayesian approaches to parameter inversion

This notebook is a short training material with some background on retrieving parameters of interest from satellite data. Within this context, the parameters of interest represent magnitudes of things that we could measure *in situ* and that are of interest as they describe the state of the land surface.

We wish to *infer* these magnitudes from indirect measurements of the spectral properties of the surface (e.g. reflectance at different wavelengths in the optical domain) and/or the magnitude of the backscatter in the microwave domain (using Synthetic Aperture Radar, SAR, data).

The parameters of interest include:
* Leaf area index (LAI): amount of one sided leaf area per unit area of ground $[m^{2}\cdot m^{2}]$
* Leaf pigment concentrations, such as the concentration of chlorophyll, water and senescent vegetation in the leaves
* Soil moisture content
* Soil roughness

## The data sources

### Optical data
Optical data (e.g. from Sentinel 2 or Landsat sensors) provides measurements of surface reflectance at a number of discrete spectral bands covering the solar reflective domain (from 400 to 2500 nm, which includes the visible part of the spectrum, from around 400 to 700 nm). The optical signal is obscured by clouds and night. We have a number of physical models (based on radiative transfer theory) that relate the surface parameters to the reflectance as seen by the sensor. Mathematically, and denoting the acquisition geometry by $\Omega, \Omega'$, the wavelengths of the sensor by $\lambda_i$ and calling the surface reflectance $\rho$:

$$
\mathcal{H}(LAI, C_{ab}, C_{w}, \dots, \Omega, \Omega') = \rho(\lambda_i, \Omega, \Omega')
$$

The problem is that we measure the stuff on the right hand side, but want to figure out the parameters in the left hand side, so we want to use the model $\mathcal{H}$ "backwards". This is called the "inverse problem".


### SAR data

SAR data illuminate the land surface with a microwave beam and measure the strength of the returns. This is measurement is called backscatter, and is often measured in different polarisations. In this project, we only consider the Sentinel 1 sensor, which operates with a frequency of around 5 GHz in the so-called C band, and measures backscatter in VV and VH polarisations. So each pixel in a Sentinel 1 image has two "bands": VV and VH. In the same way that we have physical models for the optical, we also have them for predicting SAR backscatter in the $pq$ polarisation, $\sigma^{0}_{pq}$:

$$
\mathcal{H}(sm, k, LAI, \dots, \theta)=\sigma^{0}_{pq}(\theta)
$$

Again, we are faced with the problem of how to go from the right hand side to the left hand side.

It gets even worse: due to the limitations of sensors, there is an extra term on the right hand side, which is observational noise. It is also important to understand that the physical have differential sensitivity to different parameters, so one or two parameters may be have an important effect in the signal, whereas other might not have a measurable impact.



## Inverse problems

Let's introduce some common notation. The parameters of interest will be stacked in a vector $\vec{x}$. If we consider a single pixel at a single date, we have that we can have a set of observations of e.g. reflectance and/or backscatter, which we'll stack in a vector $\vec{y}$. The physical model we'll continue calling $\mathcal{H}$ if it's nonlinear (the usual case) or $\mathbf{H}$ if it's linear (not uncommon, but you may be able to locally linearise $\mathcal{H}$ via e.g. a Taylor series). 

The *measurement equations*, assuming our observations are contaminated by additive zero-mean Gaussian noise characterised by a covariance matrix $\mathbf{C}_{obs}^{-1}$ is

$$
\mathcal{H}(\vec{x})=\vec{y} + \mathcal{N}(0, \mathbf{C}_{obs}^{-1}),
$$

Or, in other words, the difference $\mathcal{H}(\vec{x}) - \vec{y})$ is a [multivariate Gaussian distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Density_function):

$$
f_{\mathbf X}(x_1,\ldots,x_k) = \frac{\exp\left(-\frac 1 2 ({\mathbf x}-{\boldsymbol\mu})^\mathrm{T}{\boldsymbol C}^{-1}_{obs}({\mathbf x}-{\boldsymbol\mu})\right)}{\sqrt{(2\pi)^k|\boldsymbol C_{obs}|}}.
$$
Assuming $\mu=0$, and taking logs to simplify things, and ignoring an unimportant additive term (e.g. the the best solution would be one that minimises

$$
J_{obs}(\vec{x}) = \frac{1}{2}\left[\mathcal{H}(\vec{x}) - \vec{y}\right]^{\top}\mathbf{C}_{obs}^{-1}\left[\mathcal{H}(\vec{x}) - \vec{y}\right]
$$

We can minimise this with a non-linear solver. However, due to noise and limited sensitivity of $\vec{y}$ to $\vec{x}$ or parts of it, the solution is usually noisy or not acceptable. A natural way to constrain the solution is to recognise the problem we have solved as a way of maximising the log-likelihood (or minimising the negative log likelihood rather but you get my drift...), and turning it into a Bayesian problem, by extending it to have a parameter prior PDF. We will assume further than the prior PDF is also multivariate Gaussian, with a mean $\vec{\mu}_{x}$ and covariance $\mathbf{C}_{prior}$. This results in $J_{prior}(\vec{x})$, by similar arguments as above

$$
J_{prior} = \frac{1}{2}\left[\vec{x} - \vec{\mu}_{x}\right]^{\top}\mathbf{C}_{prior}^{-1}\left[\vec{x} - \vec{\mu}_{x}\right].
$$

Combining them into the *a posteriori* pdf, we have a combined cost fuction given by

$$
\begin{align}
J(\vec{x}) &= J_{obs}(\vec{x})+   J_{prior}(\vec{x}) \\
&= \frac{1}{2}\left[\vec{x} - \vec{\mu}_{x}\right]^{\top}\mathbf{C}_{prior}^{-1}\left[\vec{x} - \vec{\mu}_{x}\right]+\frac{1}{2}\left[\mathcal{H}(\vec{x}) - \vec{y}\right]^{\top}\mathbf{C}_{obs}^{-1}\left[\mathcal{H}(\vec{x}) - \vec{y}\right]\\
\end{align}
$$

If we assume that $\mathcal{H}$ isn't very non-linear, we can assume that the posterior pdf is also Gaussian, with a mean given by the minimum of $J(\vec{x})$ and an associated covariance matrix given by the inverse of the Hessian (matrix of second order derivatives) of $J(\vec{x})$ around the minimum.

In essence, this is the what we are after: minimising some cost function, and in order to do that efficiently, exploting Newton or quasi-Newton minimisation methods, that require access to the gradient (Jacobian) of $J(\vec{x})$, $\nabla J$.

## Dynamical models

The parameters that we mentioned above (LAI, soil moisture) tend to vary over time (if it rains, soil moisture tends to increase, LAI varies over the growing season, ...). It is also sensible to assume that some of these parameters change *slowly* (e.g. LAI changes little day by day). Understanding the scale of temporal dynamics of the parameters allows us to combine evidence to strengthen our inference on the parameters. For example: if the LAI is assumed constant over a week, and we have 4 observations within that week, we can combine the incomplete estimates (e.g. solve four individual minimisation problems and combine the results weighted by the uncertainty of each one).

Understanding this leads on to strategies to solve problems.



# SAR and optical retrieval

In KaSKA, Sentinel 2 data are used to retrieve a time series of S2 parameters. We think that we can use these as a prior for the Sentinel 1 inversion. The Sentinel 1 physical model is the water cloud model (WCM), that basically has two contributions to the measured signal: scatter from the canopy plus a contribution from the underlyign soil attenuated by the canopy. The parameters are given in `LMU_all_fields.ipynb`, but
$$
\sigma_{pq}^{0} = A\cdot V_{1}\left[1 - \exp\left(-\frac{-2B\cdot V_{2}}{\cos\theta}\right)\right] + \exp\left(-\frac{-2B\cdot V_{2}}{\cos\theta}\right)\cdot\left(\sigma_{soil}(pq,sm,k,\theta)\right),
$$
where $A$ and $B$ are some empirical constants, $\theta$ is the angle of incidence (given by the data product), $\sigma_{soil}$ is the soil backscatter (usually calculated as a function of polarisation $pq$, angle of incidence, soil moisture and soil roughness), and $V_1$ and $V_2$ describe the canopy optical thickness, and are usually parameterised by LAI.

We develop a retrieval strategy that aims to exploit the following information:

1. We have a reasonable guess at LAI from the S2 data, so that $V_1=V_2=LAI$
2. LAI should evolve smoothly,
3. $A$ and $B$ parameters relate to canopy structure and should either vary smoothly or could be fixed for a pixel in time. They're given per polarisation, eg $A_{VV}$ and $A_{VH}$
4. A similar argument holds for $k$ (soil roughness)
5. $sm$ (soil moisture) is probably not smoothly varying in time
6. We can also simplify $\sigma_{soil}$ to be $C\cdot sm$, where $C$ is polarisation dependent and constant over time, and
7. have an extra term $D$ to remove biases etc.

This leads us to the following $\vec{x}=\left[A_{VV}, B_{VV}, C_{VV}, A_{VH}, B_{VH}, C_{VH}, D_{VH}, LAI_{1},\dots,LAI_{N_Obs},sm_{1}, \dots,sm_{N_Obs}\right]^{\top}$, where we are solving for all these parameters using data over a temporal window from $t=1$ to $t=N_{Obs}$. We can put prior PDFs on parameters, but we can encode smoothness by adding a smoothness constraint:

$$
J_{smooth}(\vec{x})=\frac{1}{2}\gamma\displaystyle{\sum_{k=1}^{N_{Obs}-1}\left[x_{k}-x_{k+1}\right]^2}
$$

In the light of the above, it is interesting to consider the cost function that we use in the example notebook:

```python
def cost_function(x, svh, svv, theta, gamma, prior_mean, prior_unc, unc=0.8):
    """A combined cost function that calls the prior, fit to the observations
    """
    # Fit to the observations
    cost1, dcost1 = cost_obs(x, svh, svv, theta, unc=unc)
    # Fit to the prior
    cost2, dcost2 = cost_prior(x, svh, svv, theta, prior_mean, prior_unc)
    # Smooth evolution of LAI
    n_obs = len(svv)
    lai = x[(6 + n_obs) :]
    cost3, dcost3 = cost_smooth(lai, gamma)
    tmp = np.zeros_like(dcost1)
    tmp[(7 + n_obs) : -1] = dcost3
    return cost1 + cost2 + cost3, dcost1 + dcost2 + tmp
```

I have aimed to transplant the $J(\vec{x}) = J_{obs}(\vec{x})+   J_{prior}(\vec{x}) + J_{smooth}(\vec{x})$ concept here. Things to note are that there's only smoothing of LAI.

# Clever clogs extensions

The problem we're solving requires an expensive non-linear minimisation only because $\mathcal{H}$ is non-linear. We can encode the smoother as a first order differences matrix $\Delta$:

$$
\begin{align}
J(\vec{x}) &= J_{obs}(\vec{x})+   J_{prior}(\vec{x}) + J_{smooth}(\vec{x}) \\
&= \frac{1}{2}\left[\vec{x} - \vec{\mu}_{x}\right]^{\top}\mathbf{C}_{prior}^{-1}\left[\vec{x} - \vec{\mu}_{x}\right]\\
&+\frac{1}{2}\left[\mathcal{H}(\vec{x}) - \vec{y}\right]^{\top}\mathbf{C}_{obs}^{-1}\left[\mathcal{H}(\vec{x}) - \vec{y}\right]\\
&+\frac{1}{2}\gamma\vec{x}^{\top}\Delta^{\top}\Delta\vec{x}\\
\end{align}
$$

Assume that we can linearise $\mathcal{H}$ around our e.g. prior mean, then the whole problem can be written linearly. A first order Taylor approximation to $\mathcal{H}$:
$$
\mathcal{H}(\vec{x}_0 + \vec{x}) \sim \mathcal{H}(\vec{x}_0) + \frac{\overbrace{\partial \mathcal{H}(\vec{x}_0)}^{\mathbf{H}'(\vec{x})}}{\partial \vec{x}}\cdot \left(\vec{x}_0 - \vec{x}\right)
$$
           
           
Bayesian inference problem necessitates calculating log-posterior as
$$
\begin{aligned}
 \left[\mathcal{H}_0 + \mathbf{H}'(\vec{x} - \vec{x}_0  ) - \vec{y} \right]^{\top}\mathbf{C}_{obs}^{-1}\left[\mathcal{H}_0 + \mathbf{H}'(\vec{x}_0 - \vec{x}) - \vec{y} \right]& \\
+& \left[\vec{x} - \vec{\mu}_{x} \right]^{\top}\mathbf{C}^{-1}_{prior}\left[\vec{x} - \vec{\mu}_{x} \right]\\
+& \gamma\cdot \Delta^{\top}\Delta\vec{x}\\
\end{aligned}
$$

Re-arranging terms and $\vec{r}=\vec{y} - \mathcal{H}_0 + \mathbf{H}'\vec{x}_0$
$$
\overbrace{\left[\mathbf{H}'^{\top}\mathbf{C}_{obs}^{-1}\mathbf{H}' + \mathbf{C}_{prior}^{-1} + \Delta^{\top}\Delta \right]}^{\mathbf{A}}\vec{x} =  \underbrace{\mathbf{H}'^{\top}\mathbf{C}_{obs}^{-1}\vec{r} +  \mathbf{C}_{prior}^{-1}\vec{\mu}_{x}}_{\vec{b}}
$$

Now that looks pretty horrible, but this is a linear problem, where the $\mathbf{A}$ matrix is block diagonal (efficient to invert). 